In [2]:
import csv
import os
import requests as req
import json
from collections import Counter

In [3]:
#Donorschoose.org API
api_key = "DONORSCHOOSE"
url = "https://api.donorschoose.org/common/json_feed.html?max=100"

In [4]:
#Open a CSV for a list of Bay Area cities
with open("bayarea_cities.csv") as cities:
    cities_reader = csv.reader(cities)
    
    cities = [city[0] for city in cities_reader]
    
    print(len(cities))

cities = cities[94:102]
print(cities)

102
['Tiburon', 'Union City', 'Vacaville', 'Vallejo', 'Walnut Creek', 'Windsor', 'Woodside', 'Yountville']


In [5]:
# Build partial query URL
query_url = url + "&APIkey=" + api_key + "&state=CA" + "&cityName="
#location_data = [req.get(query_url + city).json() for city in cities]

all_data = {}

for city in cities:
    
    results = req.get(query_url + city).json()
    all_data[city] = results   

print(json.dumps(all_data, indent=2))

{
  "Tiburon": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Tiburon&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "0",
    "index": "0",
    "max": "50",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": []
  },
  "Union City": {
    "searchTerms": "CA",
    "searchURL": "https://www.donorschoose.org/donors/search.html?APIkey=DONORSCHOOSE&cityName=Union+City&state=CA&utm_source=api&utm_medium=feed&utm_content=searchlink&utm_campaign=DONORSCHOOSE",
    "totalProposals": "25",
    "index": "0",
    "max": "25",
    "breadcrumb": [
      [
        "max",
        "100",
        ""
      ],
      [
        "state",
        "CA",
        "California"
      ]
    ],
    "proposals": [
      {
        "id": "2889247",
        "hashCod

In [6]:
city_data = {}

for cityname, data in all_data.items():
        
    #step 1: get cost to complete
    #step 2: get povertyType label
    #step 3: add this to the output vairable
    
    costcomplete = []
    povertytype = []
    
    data["proposals"]
    
    for x in data["proposals"]:
        
        costcomplete.append(float(x["costToComplete"]))
        povertytype.append(x["povertyType"]["label"])
    
    city_data[cityname] = {
        
        "costToComplete": sum(costcomplete),
        "povertyType": Counter(povertytype)
    }

city_data

{'Tiburon': {'costToComplete': 0, 'povertyType': Counter()},
 'Union City': {'costToComplete': 33317.36,
  'povertyType': Counter({'LOW': 1,
           'MODERATE': 10,
           'MODERATEHIGH': 7,
           'MODERATELOW': 7})},
 'Vacaville': {'costToComplete': 909.74,
  'povertyType': Counter({'HIGH': 1, 'MODERATELOW': 1})},
 'Vallejo': {'costToComplete': 5431.39,
  'povertyType': Counter({'HIGH': 3,
           'HIGHEST': 5,
           'LOW': 1,
           'MODERATEHIGH': 3})},
 'Walnut Creek': {'costToComplete': 48571.03999999999,
  'povertyType': Counter({'LOW': 50})},
 'Windsor': {'costToComplete': 0, 'povertyType': Counter()},
 'Woodside': {'costToComplete': 0, 'povertyType': Counter()},
 'Yountville': {'costToComplete': 0, 'povertyType': Counter()}}

In [7]:
#list of cityname, costtocomplete, number of low, number of modeatelow, etc.

csv_data = []

citynames = sorted(city_data.keys())

for cityname in citynames: 
    
    data = city_data[cityname]
    
    one_row = []
    
    one_row.append(cityname)
    one_row.append(data['costToComplete'])  
    
    labeltype = ["LOW", "MODERATELOW", "MODERATE", "MODERATEHIGH", "HIGH", "HIGHEST", "UNKNOWN"]
    
    for x in labeltype:
        
        one_row.append(data['povertyType'].get(x, 0))
    
    csv_data.append(one_row)

csv_data

[['Tiburon', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Union City', 33317.36, 1, 7, 10, 7, 0, 0, 0],
 ['Vacaville', 909.74, 0, 1, 0, 0, 1, 0, 0],
 ['Vallejo', 5431.39, 1, 0, 0, 3, 3, 5, 0],
 ['Walnut Creek', 48571.03999999999, 50, 0, 0, 0, 0, 0, 0],
 ['Windsor', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Woodside', 0, 0, 0, 0, 0, 0, 0, 0],
 ['Yountville', 0, 0, 0, 0, 0, 0, 0, 0]]

In [8]:
root_path = os.path.join(os.getcwd())
data_path = os.path.join(root_path)
csvpath = os.path.join(root_path, "donorschoose_api.csv")
with open(csvpath, "a", newline="") as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=",")
    csvwriter.writerows(csv_data)